# * VINSIGHT : Special Metric

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Revenue Corporate (C, P)

### Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-05, 10:08:06

DataFrame: 13536 rows, 13 columns


In [3]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [4]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

In [5]:
# ''' Sample Data '''

# sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# sample_mth_df

### Peparing Check Sum Data

In [6]:
''' B0R000100 : Total Revenue '''

v_metric_cd = 'B0R000100'
v_metric_name = 'Total Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_B0R000100_df = monthly_df.copy()
chk_B0R000100_df = chk_B0R000100_df.loc[chk_B0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000100_df['METRIC_CD'] = v_metric_cd
chk_B0R000100_df['METRIC_NAME'] = v_metric_name
chk_B0R000100_df = chk_B0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000100_df = chk_B0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000100_df_display = chk_B0R000100_df.copy()
mod_col_list = chk_B0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000100_df_display[col] = chk_B0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,B0R000100,Total Revenue (sum),"12,527,775,340","12,494,707,100"
1,202402,B0R000100,Total Revenue (sum),"12,248,043,279","12,224,536,694"
2,202403,B0R000100,Total Revenue (sum),"12,484,397,821","12,459,268,326"
3,202404,B0R000100,Total Revenue (sum),"12,358,592,798","12,334,721,853"
4,202405,B0R000100,Total Revenue (sum),"12,465,806,354","12,440,265,912"
5,202406,B0R000100,Total Revenue (sum),"12,371,723,897","12,347,976,614"
6,202407,B0R000100,Total Revenue (sum),"12,489,879,839","12,463,315,685"
7,202408,B0R000100,Total Revenue (sum),"12,624,587,918","12,594,999,452"
8,202409,B0R000100,Total Revenue (sum),"12,510,022,981","12,479,579,804"
9,202410,B0R000100,Total Revenue (sum),"12,569,510,549","12,523,605,918"


In [7]:
''' TB0R000100 : Total Revenue : TMH '''

v_metric_cd = 'TB0R000100'
v_metric_name = 'Total Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_TB0R000100_df = monthly_df.copy()
chk_TB0R000100_df = chk_TB0R000100_df.loc[chk_TB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000100_df['METRIC_CD'] = v_metric_cd
chk_TB0R000100_df['METRIC_NAME'] = v_metric_name
chk_TB0R000100_df = chk_TB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000100_df = chk_TB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000100_df_display = chk_TB0R000100_df.copy()
mod_col_list = chk_TB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000100_df_display[col] = chk_TB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,TB0R000100,Total Revenue : TMH (sum),"7,813,099,416","7,779,918,671"
1,202402,TB0R000100,Total Revenue : TMH (sum),"7,646,895,055","7,623,278,040"
2,202403,TB0R000100,Total Revenue : TMH (sum),"7,791,701,211","7,766,484,467"
3,202404,TB0R000100,Total Revenue : TMH (sum),"7,735,437,287","7,711,475,155"
4,202405,TB0R000100,Total Revenue : TMH (sum),"7,848,220,835","7,822,600,234"
5,202406,TB0R000100,Total Revenue : TMH (sum),"7,820,754,103","7,796,921,421"
6,202407,TB0R000100,Total Revenue : TMH (sum),"7,873,107,450","7,846,441,194"
7,202408,TB0R000100,Total Revenue : TMH (sum),"8,025,159,126","7,995,465,389"
8,202409,TB0R000100,Total Revenue : TMH (sum),"7,987,969,545","7,957,442,638"
9,202410,TB0R000100,Total Revenue : TMH (sum),"7,986,466,416","7,940,466,913"


In [8]:
''' DB0R000100 : Total Revenue : DTAC '''

v_metric_cd = 'DB0R000100'
v_metric_name = 'Total Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000100_df = monthly_df.copy()
chk_DB0R000100_df = chk_DB0R000100_df.loc[chk_DB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000100_df['METRIC_CD'] = v_metric_cd
chk_DB0R000100_df['METRIC_NAME'] = v_metric_name
chk_DB0R000100_df = chk_DB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000100_df = chk_DB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000100_df_display = chk_DB0R000100_df.copy()
mod_col_list = chk_DB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000100_df_display[col] = chk_DB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,DB0R000100,Total Revenue : DTAC (sum),"4,714,675,924","4,714,788,429"
1,202402,DB0R000100,Total Revenue : DTAC (sum),"4,601,148,224","4,601,258,654"
2,202403,DB0R000100,Total Revenue : DTAC (sum),"4,692,696,611","4,692,783,860"
3,202404,DB0R000100,Total Revenue : DTAC (sum),"4,623,155,511","4,623,246,698"
4,202405,DB0R000100,Total Revenue : DTAC (sum),"4,617,585,519","4,617,665,679"
5,202406,DB0R000100,Total Revenue : DTAC (sum),"4,550,969,794","4,551,055,193"
6,202407,DB0R000100,Total Revenue : DTAC (sum),"4,616,772,389","4,616,874,491"
7,202408,DB0R000100,Total Revenue : DTAC (sum),"4,599,428,792","4,599,534,063"
8,202409,DB0R000100,Total Revenue : DTAC (sum),"4,522,053,436","4,522,137,166"
9,202410,DB0R000100,Total Revenue : DTAC (sum),"4,583,044,133","4,583,139,005"


In [9]:
''' B0R000101 : Mobile Revenue '''

v_metric_cd = 'B0R000101'
v_metric_name = 'Mobile Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_B0R000101_df = monthly_df.copy()
chk_B0R000101_df = chk_B0R000101_df.loc[chk_B0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000101_df['METRIC_CD'] = v_metric_cd
chk_B0R000101_df['METRIC_NAME'] = v_metric_name
chk_B0R000101_df = chk_B0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000101_df = chk_B0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000101_df_display = chk_B0R000101_df.copy()
mod_col_list = chk_B0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000101_df_display[col] = chk_B0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,B0R000101,Mobile Revenue (sum),"10,648,559,952","10,648,471,525"
1,202402,B0R000101,Mobile Revenue (sum),"10,369,921,251","10,369,647,933"
2,202403,B0R000101,Mobile Revenue (sum),"10,577,815,541","10,577,259,991"
3,202404,B0R000101,Mobile Revenue (sum),"10,457,198,744","10,456,013,816"
4,202405,B0R000101,Mobile Revenue (sum),"10,566,485,841","10,562,391,671"
5,202406,B0R000101,Mobile Revenue (sum),"10,474,207,738","10,470,194,088"
6,202407,B0R000101,Mobile Revenue (sum),"10,562,171,579","10,557,495,385"
7,202408,B0R000101,Mobile Revenue (sum),"10,597,185,877","10,593,008,396"
8,202409,B0R000101,Mobile Revenue (sum),"10,476,473,596","10,473,593,247"
9,202410,B0R000101,Mobile Revenue (sum),"10,617,659,509","10,613,906,451"


In [10]:
''' TB0R000101 : Mobile Revenue : TMH '''

v_metric_cd = 'TB0R000101'
v_metric_name = 'Mobile Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_TB0R000101_df = monthly_df.copy()
chk_TB0R000101_df = chk_TB0R000101_df.loc[chk_TB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000101_df['METRIC_CD'] = v_metric_cd
chk_TB0R000101_df['METRIC_NAME'] = v_metric_name
chk_TB0R000101_df = chk_TB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000101_df = chk_TB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000101_df_display = chk_TB0R000101_df.copy()
mod_col_list = chk_TB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000101_df_display[col] = chk_TB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,TB0R000101,Mobile Revenue : TMH (sum),"5,933,884,028","5,933,683,096"
1,202402,TB0R000101,Mobile Revenue : TMH (sum),"5,768,773,028","5,768,389,279"
2,202403,TB0R000101,Mobile Revenue : TMH (sum),"5,885,118,930","5,884,476,131"
3,202404,TB0R000101,Mobile Revenue : TMH (sum),"5,834,043,232","5,832,767,118"
4,202405,TB0R000101,Mobile Revenue : TMH (sum),"5,948,900,322","5,944,725,992"
5,202406,TB0R000101,Mobile Revenue : TMH (sum),"5,923,237,944","5,919,138,895"
6,202407,TB0R000101,Mobile Revenue : TMH (sum),"5,945,399,190","5,940,620,894"
7,202408,TB0R000101,Mobile Revenue : TMH (sum),"5,997,757,085","5,993,474,334"
8,202409,TB0R000101,Mobile Revenue : TMH (sum),"5,954,420,160","5,951,456,081"
9,202410,TB0R000101,Mobile Revenue : TMH (sum),"6,034,615,376","6,030,767,446"


In [11]:
''' DB0R000101 : Mobile Revenue : DTAC '''

v_metric_cd = 'DB0R000101'
v_metric_name = 'Mobile Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000101_df = monthly_df.copy()
chk_DB0R000101_df = chk_DB0R000101_df.loc[chk_DB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000101_df['METRIC_CD'] = v_metric_cd
chk_DB0R000101_df['METRIC_NAME'] = v_metric_name
chk_DB0R000101_df = chk_DB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000101_df = chk_DB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000101_df_display = chk_DB0R000101_df.copy()
mod_col_list = chk_DB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000101_df_display[col] = chk_DB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202401,DB0R000101,Mobile Revenue : DTAC (sum),"4,714,675,924","4,714,788,429"
1,202402,DB0R000101,Mobile Revenue : DTAC (sum),"4,601,148,224","4,601,258,654"
2,202403,DB0R000101,Mobile Revenue : DTAC (sum),"4,692,696,611","4,692,783,860"
3,202404,DB0R000101,Mobile Revenue : DTAC (sum),"4,623,155,511","4,623,246,698"
4,202405,DB0R000101,Mobile Revenue : DTAC (sum),"4,617,585,519","4,617,665,679"
5,202406,DB0R000101,Mobile Revenue : DTAC (sum),"4,550,969,794","4,551,055,193"
6,202407,DB0R000101,Mobile Revenue : DTAC (sum),"4,616,772,389","4,616,874,491"
7,202408,DB0R000101,Mobile Revenue : DTAC (sum),"4,599,428,792","4,599,534,063"
8,202409,DB0R000101,Mobile Revenue : DTAC (sum),"4,522,053,436","4,522,137,166"
9,202410,DB0R000101,Mobile Revenue : DTAC (sum),"4,583,044,133","4,583,139,005"


In [12]:
''' Concat All Check Sum Data '''

sum_rev_corp_df = pd.concat([chk_B0R000100_df, chk_TB0R000100_df, chk_DB0R000100_df, chk_B0R000101_df, chk_TB0R000101_df, chk_DB0R000101_df])
# sum_rev_corp_df

### Comparison Overview

In [13]:
''' Total Revenue & Mobile Revenue '''

v_metric_list = [
    'B0R000100' # Total Revenue
    , 'DB0R000100' # Total Revenue : DTAC
    , 'TB0R000100' # Total Revenue : TRUE
    , 'B0R000101' # Mobile Revenue
    , 'DB0R000101' # Mobile Revenue : DTAC
    , 'TB0R000101' # Mobile Revenue : TMH
    ]

vis_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C', 'P']].copy() #, 'DIFF (C-P)'
vis_rev_corp_df = vis_rev_corp_df.loc[vis_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

merge_comparison_df = pd.merge(vis_rev_corp_df, sum_rev_corp_df[['TM_KEY_MTH', 'METRIC_CD', 'C (Sum)', 'P (Sum)']], how='left', on=['TM_KEY_MTH', 'METRIC_CD'])
merge_comparison_df['DIFF_C'] = merge_comparison_df['C'] - merge_comparison_df['C (Sum)']
merge_comparison_df['DIFF_P'] = merge_comparison_df['P'] - merge_comparison_df['P (Sum)']

comparison_df = merge_comparison_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)

comparison_df_display = comparison_df.copy()
mod_col_list = comparison_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    comparison_df_display[col] = comparison_df_display[col].apply(lambda x: format(x, ',.0f'))

comparison_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
0,202401,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"15,469,491,494","12,494,707,100","12,527,775,340","12,494,707,100","2,941,716,154",0
1,202401,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"7,656,392,078","4,714,788,429","4,714,675,924","4,714,788,429","2,941,716,154",0
2,202401,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"7,813,099,416","7,779,918,671","7,813,099,416","7,779,918,671",-0,0
3,202401,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"13,590,276,107","10,648,471,525","10,648,559,952","10,648,471,525","2,941,716,154",0
4,202401,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"7,656,392,078","4,714,788,429","4,714,675,924","4,714,788,429","2,941,716,154",0
...,...,...,...,...,...,...,...,...,...,...,...
103,202506,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,0,0,0,0,0,0
104,202506,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"478,433,118","474,984,262","478,433,118","474,984,262",0,-0
105,202506,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"261,274,102","260,807,837","261,274,102","260,807,837",0,0
106,202506,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,0,0,0,0,0,0


### Comparison by Month

In [14]:
y202501_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202501]
y202501_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
72,202501,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"15,530,009,361","12,636,945,503","12,770,106,773","12,636,945,503","2,759,902,588",0
73,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
74,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"8,190,378,845","8,057,003,936","8,190,378,845","8,057,003,936",0,0
75,202501,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"13,515,636,290","10,751,551,183","10,755,733,702","10,751,551,183","2,759,902,588",0
76,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
77,202501,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"6,176,005,774","6,171,609,616","6,176,005,774","6,171,609,616",0,0


In [15]:
y202502_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202502]
y202502_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
78,202502,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"14,637,318,577","12,212,436,891","12,236,601,216","12,212,436,891","2,400,717,361",0
79,202502,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"6,776,726,890","4,376,181,618","4,376,009,529","4,376,181,618","2,400,717,361",0
80,202502,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"7,860,591,688","7,836,255,273","7,860,591,688","7,836,255,273",0,0
81,202502,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"12,720,416,836","10,314,001,696","10,319,699,474","10,314,001,696","2,400,717,361",0
82,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"6,776,726,890","4,376,181,618","4,376,009,529","4,376,181,618","2,400,717,361",0
83,202502,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"5,943,689,946","5,937,820,079","5,943,689,946","5,937,820,079",0,0


In [16]:
y202503_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202503]
y202503_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
84,202503,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"15,251,112,086","12,587,616,315","12,516,306,281","12,587,616,315","2,734,805,804",0
85,202503,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"7,245,126,495","4,510,402,170","4,510,320,691","4,510,402,170","2,734,805,804",0
86,202503,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"8,005,985,591","8,077,214,145","8,005,985,591","8,077,214,145",0,0
87,202503,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"13,420,051,397","10,678,120,361","10,685,245,593","10,678,120,361","2,734,805,804",0
88,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"7,245,126,495","4,510,402,170","4,510,320,691","4,510,402,170","2,734,805,804",0
89,202503,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"6,174,924,902","6,167,718,192","6,174,924,902","6,167,718,192",0,0


In [17]:
y202504_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202504]
y202504_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
90,202504,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"12,526,105,797","12,501,905,390","12,526,105,797","12,501,905,390",0,0
91,202504,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"4,458,117,515","4,458,129,472","4,458,117,515","4,458,129,472",-0,0
92,202504,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"8,067,988,282","8,043,775,917","8,067,988,282","8,043,775,917",0,0
93,202504,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"10,592,109,386","10,585,358,700","10,592,109,386","10,585,358,700",0,0
94,202504,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"4,458,117,515","4,458,129,472","4,458,117,515","4,458,129,472",-0,0
95,202504,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"6,133,991,871","6,127,229,228","6,133,991,871","6,127,229,228",0,0


In [18]:
y202505_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202505]
y202505_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
96,202505,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"12,473,243,301","12,442,754,986","12,473,243,301","12,442,754,986",0,0
97,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"4,360,893,566","4,360,899,792","4,360,893,566","4,360,899,792",0,0
98,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"8,112,349,735","8,081,855,194","8,112,349,735","8,081,855,194",0,0
99,202505,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"10,550,624,069","10,536,363,812","10,550,624,069","10,536,363,812",0,0
100,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"4,360,893,566","4,360,899,792","4,360,893,566","4,360,899,792",-0,0
101,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"6,189,730,503","6,175,464,020","6,189,730,503","6,175,464,020",0,0


In [19]:
y202506_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202506]
y202506_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
102,202506,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"478,433,118","474,984,262","478,433,118","474,984,262",0,-0
103,202506,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,0,0,0,0,0,0
104,202506,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"478,433,118","474,984,262","478,433,118","474,984,262",0,-0
105,202506,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"261,274,102","260,807,837","261,274,102","260,807,837",0,0
106,202506,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,0,0,0,0,0,0
107,202506,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"261,274,102","260,807,837","261,274,102","260,807,837",0,0


### Pending...

In [20]:
# ''' B0R000100 : Total Revenue '''

# v_total_metric_list = [
#     'DB1R000100' # Prepaid Revenue : DTAC
#     , 'TB1R000100' # Prepaid Revenue : TMH
#     , 'DB2R000100' # Postpaid Revenue : DTAC
#     , 'TB2R000100' # Postpaid Revenue : TMH
#     , 'TB3R000100' # TOL Revenue
#     , 'TB4R000100' # TVS Revenue
#     ]

# B0R000100_day_df = chk_src_df.copy()
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]

# # B0R000100_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['C'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['C'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['C'], 0)

# B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'PPN_TM':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
# B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

# B0R000100_day_df['SUM (T+D)'] = B0R000100_day_df['TRUE'] + B0R000100_day_df['DTAC']
# B0R000100_day_df['DIFF (SUM-ALL)'] = B0R000100_day_df['SUM (T+D)'] - B0R000100_day_df['ALL']

# mod_col_list = B0R000100_day_df.iloc[:, 3:8].columns.tolist()
# for col in mod_col_list:
#     B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))

# B0R000100_day_df